In [1]:
from qiskit import Aer, QuantumCircuit, transpile, execute
from qiskit.circuit import Parameter
import numpy as np
from scipy.optimize import minimize
from qiskit.visualization import plot_histogram
from Target_Distibution import TARGET_FUNCTION
import matplotlib.pyplot as plt

### Define the QCBM Circuit

In [2]:
def qcbm_circuit(params, num_qubits, num_layers=2):
    """
    Construct a QCBM circuit with the specified number of layers (d).
    
    Args:
        params (np.ndarray): Array of parameters (size: num_qubits * 3 * num_layers).
        num_qubits (int): Number of qubits.
        num_layers (int): Number of layers (d).
        
    Returns:
        QuantumCircuit: The parameterized QCBM circuit.
    """
    qc = QuantumCircuit(num_qubits)
    idx = 0

    for layer in range(num_layers):
        # Single-qubit rotations for each qubit in the layer
        for q in range(num_qubits):
            qc.rz(params[idx], q)
            qc.rx(params[idx + 1], q)
            qc.rz(params[idx + 2], q)
            idx += 3
        # Add entangling gates
        for q in range(num_qubits - 1):
            qc.cx(q, q + 1)
    return qc


### Define the Loss Function 

In [3]:
def loss_function(params, num_qubits, num_layers, target_distribution):
    """
    Calculate the KL divergence between target and generated distributions.
    
    Args:
        params (np.ndarray): Current parameters of the QCBM circuit.
        num_qubits (int): Number of qubits.
        num_layers (int): Number of layers (d).
        target_distribution (dict): Target probability distribution.
        
    Returns:
        float: KL divergence.
    """
    qc = qcbm_circuit(params, num_qubits, num_layers)
    qc.measure_all()

    # Simulate the circuit
    simulator = Aer.get_backend('qasm_simulator')
    qc = transpile(qc, simulator)
    result = execute(qc, backend=simulator, shots=8000).result()
    counts = result.get_counts()
    
    # Convert counts to probabilities
    generated_distribution = {k: v / 8000 for k, v in counts.items()}
    
    # Calculate KL divergence
    kl_divergence = 0
    for bitstring, target_prob in target_distribution.items():
        generated_prob = generated_distribution.get(bitstring, 1e-10)
        kl_divergence += target_prob * np.log(target_prob / generated_prob)
    
    return kl_divergence


### Define/Compute Gradient

In [4]:
def gradient_computer(params,params_shift,num_qubits,  num_layers, target_distribution):
    qc_a = qcbm_circuit(params, num_qubits, num_layers)
    qc_a.measure_all()

    simulator = Aer.get_backend('qasm_simulator')
    qc_a = transpile(qc_a, simulator)
    result_a = execute(qc_a, backend=simulator, shots=8000).result()
    counts_a = result_a.get_counts()
    
    # Normalize counts to probabilities
    generated_distribution_a = {k: v / 8000 for k, v in counts_a.items()}
    
    qc_b = qcbm_circuit(params_shift, num_qubits, num_layers)
    qc_b.measure_all()

    simulator = Aer.get_backend('qasm_simulator')
    qc_b = transpile(qc_b, simulator)
    result_b = execute(qc_b, backend=simulator, shots=8000).result()
    counts_b = result_b.get_counts()

    # Normalize counts to probabilities
    generated_distribution_b = {k: v / 8000 for k, v in counts_b.items()}
    
    gradient_component=0
    for bitstring, target_prob in target_distribution.items():
        generated_prob_a = generated_distribution_a.get(bitstring, 1e-10)
        generated_prob_b = generated_distribution_b.get(bitstring, 1e-10)
        gradient_component -=  target_prob /generated_prob_a*generated_prob_b
        
    return gradient_component

In [5]:
def compute_gradients(params, num_qubits, num_layers, target_distribution):
    """
    Compute gradients of the loss function using the parameter-shift rule.
    
    Args:
        params (np.ndarray): Current parameters of the QCBM circuit.
        num_qubits (int): Number of qubits.
        num_layers (int): Number of layers (d).
        target_distribution (dict): Target probability distribution.
        
    Returns:
        np.ndarray: Gradients of the loss function w.r.t. parameters.
    """
    gradients = np.zeros_like(params)
    shift = np.pi / 2  # Parameter shift value

    for i in range(len(params)):
        # Shift parameter up
        params_shifted_up = np.copy(params)
        params_shifted_up[i] += shift
        loss_up = gradient_computer(params, params_shifted_up, num_qubits, num_layers, target_distribution)

        # Shift parameter down
        params_shifted_down = np.copy(params)
        params_shifted_down[i] -= shift
        loss_down = gradient_computer(params, params_shifted_down, num_qubits, num_layers, target_distribution)

        # Compute gradient
        gradients[i] =loss_up - loss_down

    return gradients

### Gradient Descent Training

In [6]:
def gradient_descent(initial_params, num_qubits, num_layers, target_distribution, learning_rate=0.1, max_iters=100):
    """
    Perform gradient descent to optimize QCBM parameters.
    
    Args:
        initial_params (np.ndarray): Initial QCBM parameters.
        num_qubits (int): Number of qubits.
        num_layers (int): Number of layers (d).
        target_distribution (dict): Target probability distribution.
        learning_rate (float): Learning rate for parameter updates.
        max_iters (int): Maximum number of iterations.
        
    Returns:
        np.ndarray: Optimized parameters.
    """
    params = np.copy(initial_params)
    for iteration in range(max_iters):
        # Compute gradients
        gradients = compute_gradients(params, num_qubits, num_layers, target_distribution)
        
        # Update parameters
        params -= learning_rate * gradients
        
        # Compute loss for monitoring
        loss = loss_function(params, num_qubits, num_layers, target_distribution)
        print(f"Iteration {iteration + 1}, Loss: {loss}")
        if loss<0.1:
            break
    
    return params


### Training the QCBM

In [ ]:
# Step 5: Train the QCBM
num_qubits = 6
num_layers = 2
num_params = num_qubits * 3 * num_layers
initial_params = np.random.uniform(0, 2 * np.pi, num_params)

# Define target distribution
target_distribution = TARGET_FUNCTION.gaussian(num_qubits)

# Train using gradient descent
optimized_params = gradient_descent(
    initial_params, 
    num_qubits, 
    num_layers, 
    target_distribution, 
    learning_rate=0.1, 
    max_iters=100
)


### Constuct Citcuit Using Trained Parameter

In [ ]:
trained_qcbm = qcbm_circuit(optimized_params, num_qubits, num_layers)
trained_qcbm.measure_all()

simulator = Aer.get_backend('qasm_simulator')
trained_qcbm = transpile(trained_qcbm, simulator)
result = execute(trained_qcbm, backend=simulator, shots=8000).result()
counts = result.get_counts()

print("Generated Distribution:", counts)


In [ ]:
trained_qcbm.draw(output="mpl")

In [ ]:
plot_histogram(counts)

### Compare Target Distribution & Generated Distribution

In [ ]:
# Combine and sort keys
all_keys = sorted(set(target_distribution.keys()).union(counts.keys()))

# Extract values for both distributions, setting missing ones to 0
generated_values = np.array([counts.get(key, 0) for key in all_keys])
target_values = np.array([target_distribution.get(key, 0) for key in all_keys])
generated_values = generated_values/8000

# Plot
plt.figure(figsize=(14, 7))

# Plot generated distribution as a histogram
plt.bar(all_keys, generated_values, color='blue', alpha=0.6, label='Generated Distribution')

# Plot target distribution as a line plot
plt.plot(all_keys, target_values, marker='o', linestyle='-', color='orange', label='Target Distribution')

# Add labels and title
plt.xticks(rotation=90)
plt.xlabel('Binary Strings')
plt.ylabel('Frequencies')
plt.title('Generated Distribution (Histogram) vs Target Distribution (Line Plot)')
plt.legend()

# Add grid and adjust layout
plt.grid(True)
plt.tight_layout()

# Show plot
plt.show()

### Another Method: Calculating Gradient Using Existing Function, minimize

In [ ]:
num_qubits = 6
num_layers = 2
num_params = num_qubits * 3 * num_layers
initial_params = np.random.uniform(0, 2 * np.pi, num_params)

# Define target distribution (e.g., |110⟩ with 0.8 probability, |001⟩ with 0.2 probability)
target_distribution = TARGET_FUNCTION.gaussian(num_qubits)

result_cob = minimize(loss_function, initial_params, args=(num_qubits,num_layers, target_distribution), method='COBYLA')

# Optimized parameters
optimized_params_cob = result_cob.x

In [ ]:
trained_qcbm_cob = qcbm_circuit(optimized_params_cob, num_qubits, num_layers)
trained_qcbm_cob.measure_all()
trained_qcbm_cob.draw(output="mpl")

In [ ]:
simulator = Aer.get_backend('qasm_simulator')
trained_qcbm = transpile(trained_qcbm_cob, simulator)
result = execute(trained_qcbm_cob, backend=simulator, shots=8000).result()
counts = result.get_counts()

print("Generated Distribution:", counts)

In [ ]:
plot_histogram(counts)

In [ ]:
plot_histogram(target_distribution)

In [ ]:
# Combine and sort keys
all_keys = sorted(set(target_distribution.keys()).union(counts.keys()))

# Extract values for both distributions, setting missing ones to 0
generated_values = np.array([counts.get(key, 0) for key in all_keys])
target_values = np.array([target_distribution.get(key, 0) for key in all_keys])
generated_values = generated_values/8000

# Plot
plt.figure(figsize=(14, 7))

# Plot generated distribution as a histogram
plt.bar(all_keys, generated_values, color='blue', alpha=0.6, label='Generated Distribution')

# Plot target distribution as a line plot
plt.plot(all_keys, target_values, marker='o', linestyle='-', color='orange', label='Target Distribution')

# Add labels and title
plt.xticks(rotation=90)
plt.xlabel('Binary Strings')
plt.ylabel('Frequencies')
plt.title('Generated Distribution (Histogram) vs Target Distribution (Line Plot)')
plt.legend()

# Add grid and adjust layout
plt.grid(True)
plt.tight_layout()

# Show plot
plt.show()